# Intro

This notebook is only used to train the network, the data generation is done in the python scripts that are provided in the github. In order to be able to run the whole notebook you need to have a folder called data with the files hits.txt and parameters.txt inside. To be able to save the model you need a folder named model.

```
- notebook.ipynb
- data
  - parameters.txt
  - hits.txt
- model
```

We start by doing all of our imports and defining some constants. Then we load the device in order to be able to use the GPU, and we load the dataset.



In [36]:
import tqdm
import torch
from torch import nn
import os
import math
import numpy as np
import pandas as pd
from torch import Tensor
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import Dataset, DataLoader, random_split
from timeit import default_timer as timer
from torch.nn.utils.rnn import pad_sequence
import matplotlib.pyplot as plt
# from utils import custom_collate, create_mask_src, create_output_pred_mask, sort_by_angle, load_variable_len_data, get_labels_2d, get_labels_3d, normalize_data

In [37]:
DETECTORS = [1, 2, 3, 4, 5]
NR_DETECTORS = len(DETECTORS)
DIM = 2
DATA_FILENAME = "data/hits.txt"
LABEL_FILENAME = "data/parameters.txt"
BATCH_SIZE = 32
TEST_BATCH_SIZE = 2
PADDING_LEN_INPUT = 100
PADDING_LEN_LBL = 20
PAD_TOKEN = 50
EARLY_STOPPING = 6
LOSS_FN = nn.MSELoss(reduction='mean')
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# utility functions

The following section includes a few functions that are used later on.

In [19]:
def custom_collate(batch):
    event_ids = []
    xs, ys, zs = [], [], []
    labels = []
    labels_pad, lbl_lens = None, None

    # load in the elements in the batch
    for b in batch:
        event_ids.append(b[0])
        xs.append(b[1])
        ys.append(b[2])
        zs.append(b[3])
        labels.append(b[4])

    x_lens = [len(val) for val in xs]
    lbl_lens = [len(lbl) for lbl in labels]

    # pad the labels
    if DIM == 2:
        labels[0] = nn.ConstantPad1d((0, PADDING_LEN_LBL - labels[0].shape[0]), PAD_TOKEN)(labels[0])
    if DIM == 3:
        labels[0] = nn.ConstantPad2d((0, 0, 0, PADDING_LEN_LBL - labels[0].shape[0]), PAD_TOKEN)(labels[0])
    labels_pad = pad_sequence(labels, batch_first=False, padding_value=PAD_TOKEN)

    # add padding to the x, y and z vectors
    xs[0] = nn.ConstantPad1d((0, PADDING_LEN_INPUT - xs[0].shape[0]), PAD_TOKEN)(xs[0])
    ys[0] = nn.ConstantPad1d((0, PADDING_LEN_INPUT - ys[0].shape[0]), PAD_TOKEN)(ys[0])
    zs[0] = nn.ConstantPad1d((0, PADDING_LEN_INPUT - zs[0].shape[0]), PAD_TOKEN)(zs[0])

    xs_pad = pad_sequence(xs, batch_first=False, padding_value=PAD_TOKEN)
    ys_pad = pad_sequence(ys, batch_first=False, padding_value=PAD_TOKEN)
    zs_pad = pad_sequence(zs, batch_first=False, padding_value=PAD_TOKEN)
    x = torch.stack((xs_pad, ys_pad, zs_pad), dim=1)

    # Return the final batch
    return event_ids, x.transpose(1, 2), x_lens, labels_pad, lbl_lens


def load_variable_len_data(path):
    # from https://stackoverflow.com/questions/27020216/import-csv-with-different-number-of-columns-per-row-using-pandas
    with open(path, 'r') as f:
        col_count = [len(l.split(",")) for l in f.readlines()]

    # create column names corresponding to their index
    column_names = [i for i in range(0, max(col_count))]

    # read data with the previously created column names
    data = pd.read_csv(path, header=None, delimiter=",", names=column_names)
    return data


def create_mask_src(src, device):
    src_seq_len = src.shape[0]
    padding_vector = torch.full((src_seq_len,), PAD_TOKEN, device=device)
    src_mask = torch.zeros((src_seq_len, src_seq_len), device=device).type(torch.bool)
    src_padding_mask = (src.transpose(0, 2) == padding_vector).all(dim=0)

    return src_mask, src_padding_mask


def create_output_pred_mask(tensor, indices):
    indices_arr = np.array(indices)
    row_indices = np.arange(tensor.shape[1])[:, np.newaxis]
    col_indices = np.arange(tensor.shape[0])
    mask = col_indices < indices_arr[row_indices]
    return mask.T


def get_labels_2d(event_labels, sort=True):
    labels = event_labels[2::2]
    labels = [float(value) for value in labels if not math.isnan(value)]
    if sort:
        labels = np.sort(labels)
    # print(labels)
    return labels


def get_labels_3d(event_labels, sort=True):
    labels = event_labels[2::2]

    tmp_labels = []
    for angles in labels:
        if not isinstance(angles, float): # if there is a nan, it's a float
            angles = angles.split(';')
            tmp_labels.append((float(angles[0]), float(angles[1])))
    labels = tmp_labels

    if sort:
        labels = np.sort(labels)
    return labels


def normalize_data(data):
    maximum = data.abs().max()
    norm_data = data / maximum
    return norm_data

def cart2cyl(x, y, z=None):
    rho = np.sqrt(x ** 2 + y ** 2)
    phi = np.arctan2(y, x)
    return (rho, phi, z) if z is not None else (rho, phi)


def sort_by_angle(x, y, z = None):
    # convert the seperate lists into one list of tuples
    coords = []
    for i in range(len(x)):
            if DIM == 2:
                coords.append((x[i], y[i]))
            elif DIM == 3:
                coords.append((x[i], y[i], z[i]))

    # sort the list
    dist_coords = np.array(coords)
    distances = np.round(np.linalg.norm(dist_coords, axis=1))
    # Sort first by rho, round the rho, then sort by phi (sorting by the angle on detector)
    cylindrical_coords = [cart2cyl(*coord) for coord in coords]
    sorted_indices = np.lexsort((list(zip(*cylindrical_coords))[1], distances))
    sorted_cartesian_coords = [coords[i] for i in sorted_indices]

    # convert back into seperate lists
    if DIM == 2:
        x, y = zip(*sorted_cartesian_coords)
    elif DIM == 3:
        x, y, z = zip(*sorted_cartesian_coords)

    return x, y, z


# The Dataset
The following code block defines the dataset which is used by the data loader later on.

In [20]:
class TrajectoryDataset(Dataset):
    def __init__(self, data_filename, labels_filename, normalize=False):
        self.data = load_variable_len_data(data_filename)
        self.labels = load_variable_len_data(labels_filename)

        self.total_nr_events = len(self.data)
        self.normalize = normalize


    def __len__(self):
        return self.total_nr_events


    def __getitem__(self, idx):
        labels = None
        data = self.data.iloc[[idx]].values.tolist()[0]
        event_id = int(data[0])

        # get the label
        event_labels = self.labels.iloc[[event_id]].values.tolist()[0]
        if DIM == 2:
            labels = get_labels_2d(event_labels)
        elif DIM == 3:
            labels = get_labels_3d(event_labels)

        # get the x and y coordinates
        x = data[1::DIM + 1]
        y = data[2::DIM + 1]

        # convert to float
        x = [float(value) for value in x if not math.isnan(value)]
        y = [float(value) for value in y if not math.isnan(value)]

        # if 2d data we fill the z vector with padding, if 3d data we fill it with data
        z = None
        if DIM == 2:
            z = [PAD_TOKEN] * len(x)
        elif DIM == 3:
            z = data[3::DIM + 1]
            z = [float(value) for value in z if not math.isnan(value)]

        # normalise
        if self.normalize:
            raise NotImplementedError() #TODO

        # sort the data
        if DIM == 2:
            x, y, _ = sort_by_angle(x, y)
        elif DIM == 3:
            x, y, z = sort_by_angle(x, y, z)

        # convert the coordinates and labels to tensors
        x = torch.tensor(x).float()
        y = torch.tensor(y).float()
        z = torch.tensor(z).float()
        labels = torch.tensor(labels).float()

        # clean up data
        del data

        return event_id, x, y, z, labels

In [21]:
dataset = TrajectoryDataset(DATA_FILENAME, LABEL_FILENAME)

# Training Functions
The first function is used both for training and evaluating of the model. It gets the data and creates masks, then gets predictions from the model to calculate the loss.

In [22]:
def train_eval(model, progress_bar, train=True, optim=None):
    losses = 0.
    for i, data in progress_bar:
        #get the data and labels on the device
        _, x, src_len, labels, _ = data
        x = x.to(DEVICE)
        if labels is not None:
            labels = labels.to(DEVICE)

        # create masks for the data
        src_mask, src_padding_mask = create_mask_src(x, DEVICE)

        # run model
        pred = model(x, src_mask, src_padding_mask)
        if train:
            optim.zero_grad()

        # create and apply mask to the labels
        mask = (labels != PAD_TOKEN).float()
        padding_len = np.round(np.divide(src_len, NR_DETECTORS))
        labels = labels * mask

        # calculate loss for 2d data
        if DIM == 2:
            pred = pred.transpose(0, 1)
            pred_mask = create_output_pred_mask(pred, padding_len)
            pred = pred * torch.tensor(pred_mask, device=DEVICE).float()
            # loss calculation
            loss = LOSS_FN(pred, labels)

        # calculate loss for 3d data
        elif DIM == 3:
            pred = pred[0].transpose(0, 1), pred[1].transpose(0, 1)
            pred = torch.stack([pred[0], pred[1]])
            for slice_ind in range(pred.shape[0]):
                slice_mask = create_output_pred_mask(pred[slice_ind, :, :], padding_len)
                pred[slice_ind, :, :] = pred[slice_ind, :, :] * torch.tensor(slice_mask, device=DEVICE).float()
            pred = pred.transpose(0, 2)
            pred = pred.transpose(1, 0)
            # loss calculation
            loss = LOSS_FN(pred, labels)

        # if training, compute gradients and do backpropagation
        if train:
            loss.backward()
            optim.step()

        # update the progress bar
        progress_bar.set_description("loss = %.8f" % loss.item())
        losses += loss.item()
    return losses

In [23]:
def evaluate(model, batch_size, loader):
    model.eval()
    n_batches = int(math.floor(len(loader.dataset) / batch_size))
    progress_bar = tqdm.tqdm(enumerate(loader), total=n_batches)

    with torch.no_grad():
        losses = train_eval(model, progress_bar, train=False)

    return losses / len(loader)

In [24]:
def train(t_loader, v_loader, transformer, optimizer, batch_size):
    train_losses, val_losses = [], []
    min_val_loss = np.inf
    load = False
    epoch, count = 0, 0

    print("Starting training...")

    for epoch in range(epoch, EPOCHS):
        start_time = timer()

        # we set the model to train mode and enable gradients
        torch.set_grad_enabled(True)
        transformer.train()

        # we calculate the number of batches for the progress bar
        n_batches = int(math.floor(len(t_loader.dataset) / batch_size))
        progress_bar = tqdm.tqdm(enumerate(t_loader), total=n_batches)

        # we calculate the loss by calling the train_eval function
        losses = train_eval(transformer, progress_bar, train=True, optim=optimizer)
        train_loss = losses / len(t_loader)

        end_time = timer()

        # we evaluat the model on the validation set
        val_loss = evaluate(transformer, batch_size, v_loader)
        print("\nEpoch: ", epoch, ", Train loss: ", train_loss, \
               ", Val loss: ", val_loss, ", Epoch time: ", end_time - start_time, "\n")

        train_losses.append(train_loss)
        val_losses.append(val_loss)

        # either saving the best model or the last model and keeping track of early stopping
        if val_loss < min_val_loss:
            min_val_loss = val_loss
            print("Saving best model with val_loss: {}".format(val_loss))
            torch.save({'epoch': epoch, 'model_state_dict': transformer.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(), 'train_losses': train_losses,
                'val_losses': val_losses,
            }, "model/transformer_encoder_best")
            count = 0
        else:
            print("Saving last model with val_loss: {}".format(val_loss))
            torch.save({'epoch': epoch, 'model_state_dict': transformer.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(), 'train_losses': train_losses,
                'val_losses': val_losses,
            }, "model/transformer_encoder_last")
            count += 1

        # early stopping criterion
        if count >= EARLY_STOPPING:
            print("Early stopping")
            break
    print("Final best loss: ", min_val_loss)

# The Model

The following class describes the model itself, this code is mostly based on the paper "Artificial intelligence for improved fitting of trajectories of elementary particles in inhomogeneous dense materials immersed in a magnetic field". For which code was provided.

In [25]:
class FittingTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,  # number of Transformer encoder layers
                 d_model: int,  # length of the new representation
                 n_head: int,  # number of heads
                 input_size: int,  # size of each item in the input sequence
                 output_size: int,  # size of each item in the output sequence
                 dim_feedforward: int,  # DIM of the feedforward network of the Transformer
                 dropout: float = 0.1,  # dropout value
                 seq_len: int = 20):

        super(FittingTransformer, self).__init__()
        encoder_layers = TransformerEncoderLayer(d_model=d_model,
                                                 nhead=n_head,
                                                 dim_feedforward=dim_feedforward,
                                                 dropout=dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, num_encoder_layers)
        self.proj_input = nn.Linear(input_size, d_model)
        self.aggregator = nn.Linear(seq_len, 1)
        self.decoder_angle1 = nn.Linear(d_model, output_size)
        self.decoder_angle2 = nn.Linear(d_model, output_size)
        self.dropout = nn.Dropout(dropout)
        self.init_weights()

    def init_weights(self, init_range=0.1) -> None:
        # weights initialisation
        self.proj_input.bias.data.zero_()
        self.proj_input.weight.data.uniform_(-init_range, init_range)
        self.decoder_angle1.bias.data.zero_()
        self.decoder_angle1.weight.data.uniform_(-init_range, init_range)
        self.decoder_angle2.bias.data.zero_()
        self.decoder_angle2.weight.data.uniform_(-init_range, init_range)

    def forward(self,
                src: Tensor,
                mask: Tensor,
                src_key_padding_mask: Tensor):

        # Linear projection of the input
        src_emb = self.proj_input(src)

        # Transformer encoder
        memory = self.transformer_encoder(src=src_emb, mask=mask,
                                          src_key_padding_mask=src_key_padding_mask)
        memory = torch.mean(memory, dim=0)

        # Dropout
        memory = self.dropout(memory)

        # Linear projection of the output, with 1 output if there are 2 dimensions and 2 outputs if there are 3 dimensions
        if DIM == 2:
            output = self.decoder_angle1(memory)
            return output
        if DIM == 3:
            output1 = self.decoder_angle1(memory)
            output2 = self.decoder_angle2(memory)
            return output1, output2


# Setting up the data

We define the parameters for the model in order to easily change them for grid search. Then we split the data, create the dataloaders and initialise the model.

In [31]:
D_MODEL = 128
NUM_ENCODER_LAYERS = 6
N_HEAD = 8
DIM_FEEDFORWARD = 128
DROPOUT = 0.1
EPOCHS = 60

In [32]:
# split data into train=0.6, val=0.2 and test=0.2
proportions = [.6, .2, .2]
lengths = [int(p * len(dataset)) for p in proportions]
lengths[-1] = len(dataset) - sum(lengths[:-1])
train_set, val_set, test_set = random_split(dataset, lengths, generator=torch.Generator().manual_seed(123))

# create train, validation, and test loaders
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE,
                          num_workers=1, shuffle=True, collate_fn=custom_collate)
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE,
                          num_workers=1, shuffle=False, collate_fn=custom_collate)
test_loader = DataLoader(test_set, batch_size=TEST_BATCH_SIZE,
                          num_workers=1, shuffle=False, collate_fn=custom_collate)

# Initialise the transformer model
transformer = FittingTransformer(num_encoder_layers=NUM_ENCODER_LAYERS,
                                d_model=D_MODEL,
                                n_head=N_HEAD,
                                input_size=3,
                                output_size=20,
                                dim_feedforward=DIM_FEEDFORWARD,
                                dropout=DROPOUT)
transformer = transformer.to(DEVICE)

# print the total number of trainable parameters
pytorch_total_params = sum(p.numel() for p in transformer.parameters() if p.requires_grad)
print("Total number of trainable parameters: {}".format(pytorch_total_params))

# initialise the optimizer
optimizer = torch.optim.Adam(transformer.parameters(), lr=1e-4)

Total number of trainable parameters: 603197


We do a small sanity check before training to see if the data is what we expect it to be

In [33]:
print("Length of data: ", len(dataset.data))
print("Length of labels: ", len(dataset.labels))

print("Length of train set: ", len(train_set))
print("Length of val set: ", len(val_set))
print("Length of test set: ", len(test_set))

Length of data:  50000
Length of labels:  50000
Length of train set:  30000
Length of val set:  10000
Length of test set:  10000


#Training

Here we can finally call the train function.

In [34]:
train(train_loader, val_loader, transformer, optimizer, BATCH_SIZE)

Starting training...


loss = 0.19905722: : 938it [00:46, 20.38it/s]                       
loss = 0.16587153: : 313it [00:12, 24.49it/s]                       


Epoch:  0 , Train loss:  0.30813188290100363 , Val loss:  0.17712856964847912 , Epoch time:  46.07615964800061 

Saving last model with val_loss: 0.17712856964847912



loss = 0.18012981: : 938it [00:46, 20.30it/s]                       
loss = 0.13968848: : 313it [00:12, 24.58it/s]                       



Epoch:  1 , Train loss:  0.17522346187851576 , Val loss:  0.14608259951344693 , Epoch time:  46.2672115080004 

Saving last model with val_loss: 0.14608259951344693


...


Epoch:  7 , Train loss:  0.07934440473027067 , Val loss:  0.13937089577936135 , Epoch time:  46.02659069100082 

Saving last model with val_loss: 0.13937089577936135


loss = 0.06712691: : 938it [00:45, 20.41it/s]                       
loss = 0.19890425: : 313it [00:12, 24.90it/s]                       



Epoch:  8 , Train loss:  0.07487938146013567 , Val loss:  0.15571078822349968 , Epoch time:  46.01419160000023 

Saving last model with val_loss: 0.15571078822349968


loss = 0.10242712: : 938it [00:47, 19.71it/s]                       
loss = 0.20300360: : 313it [00:12, 24.42it/s]                       



Epoch:  9 , Train loss:  0.0691444110085588 , Val loss:  0.1560754012137937 , Epoch time:  47.64256949099945 

Saving last model with val_loss: 0.1560754012137937
Early stopping
Final best loss:  0.10860962025559368


# Loading a saved model

If the training has already been done, we can load in a saved model with the following code.

The code from setting up the data does have to be ran before this.

In [ ]:
checkpoint = torch.load("model/transformer_encoder_last")
transformer.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch'] + 1
train_losses = checkpoint['train_losses']
val_losses = checkpoint['val_losses']
min_val_loss = min(val_losses)
print(epoch, val_losses)

# Evaluating

The final step is calculating the loss over the test set.

In [35]:
test_loss = evaluate(transformer, TEST_BATCH_SIZE, test_loader)
print("\ntest loss: ", test_loss)

loss = 0.12862778: 100%|██████████| 5000/5000 [01:04<00:00, 77.40it/s]


test loss:  0.1556599835234607
